In [1]:
!pip install transformers torch

In [23]:
import torch
from transformers import pipeline

class ProductWriterAI:
    def __init__(self):
        device_id = 0 if torch.cuda.is_available() else -1
        if device_id == 0:
            print("GPU Enabled.")

        print("Loading models...")

        self.generator = pipeline("text-generation", model="distilgpt2", device=device_id)

        self.ner = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple", device=device_id)

        self.summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=device_id)

        self.qa = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", device=device_id)

    def create_marketing_package(self, product_name):

        prompt = f"The {product_name} is a premium solution designed to provide"

        generated = self.generator(
            prompt,
            max_new_tokens=60,
            num_return_sequences=1,
            temperature=0.7,
            repetition_penalty=1.3,
            do_sample=True,
            pad_token_id=50256
        )

        sales_pitch = generated[0]['generated_text']
        if "." in sales_pitch:
            sales_pitch = sales_pitch[:sales_pitch.rfind(".") + 1]

        summary = self.summarizer(sales_pitch, max_length=15, min_length=5, do_sample=False)
        tagline = summary[0]['summary_text']

        entities = self.ner(sales_pitch)

        question = "What does this product provide?"
        answer = self.qa(question=question, context=sales_pitch)

        return {
            "Product": product_name,
            "Sales Pitch": sales_pitch,
            "Tagline": tagline,
            "Entities Detected": entities,
            "FAQ": answer['answer']
        }

if __name__ == "__main__":
    ai_marketer = ProductWriterAI()

    product = input("\nEnter a product name (e.g., Luxury Fountain Pen): ")
    result = ai_marketer.create_marketing_package(product)

    print(f"\n### Marketing Copy for: {result['Product']} ###")
    print(f"\n[Generated Pitch]\n{result['Sales Pitch']}")
    print(f"\n[Generated Tagline]\n{result['Tagline']}")

    print(f"\n[NER Extraction]")
    for entity in result['Entities Detected']:
        print(f"- {entity['word']} ({entity['entity_group']})")

    print(f"\n[Automated QA]")
    print(f"Q: What is the main benefit?\nA: {result['FAQ']}")

GPU Enabled.
Loading models...


Device set to use cuda:0
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0



Enter a product name (e.g., Luxury Fountain Pen): Samsung phone

### Marketing Copy for: Samsung phone ###

[Generated Pitch]
The Samsung phone is a premium solution designed to provide unparalleled security. It's the flagship smartphone of our smartphones, and we're excited about that as well!
A Galaxy S6 edge with LTE support powered by Qualcomm Snapdragon 801 (8GB RAM) processor can be found at www://www-samsungappsstore/products.

[Generated Tagline]
 Galaxy S6 edge with LTE support powered by Qualcomm Snapdragon 8

[NER Extraction]
- Samsung (ORG)
- Galaxy S6 (MISC)
- L (MISC)
- Qualcomm Snapdragon (MISC)

[Automated QA]
Q: What is the main benefit?
A: unparalleled security
